## 2. 機械学習実践のための基礎技術
### 2.1 正確な予測を導く
#### 問題設定の導入(セグメント拡張のためのユーザ属性の予測)
広告主は、宣伝したい商品やサービスの特性に合わせて、年齢や性別を限定したうえで広告を配信したいという要望を持っている。  
この年齢や性別を限定したユーザのグループのことを**セグメント**と呼ぶ。  
ユーザの属性情報に基づいたセグメント別の広告配信がさまざまなメディアで行われている。  
しかし、Web上のメディアやサービスには基本的な属性情報さえ得られないユーザが多く存在し、セグメントを割り当てることができない。  
この事情により、属性情報を取得できるのは会員やアンケート回答者など一部のユーザに限られ、  
それ以外のユーザの属性情報を得ることはできない。   
ここに、属性情報が取得できないユーザの属性情報を予測してセグメントに割り振り、  
広告配信を実行できるユーザ数を増やすことで売り上げを増大させたいというモチベーションが生まれる。  
  
セグメント拡張とは、年齢や性別などセグメントを構築するために必要な属性情報を予測するタスクである。  
ここでは、行動ログなどを特徴量としてセグメント構築に使いたい属性情報の予測モデルを学習し、他のユーザの属性情報を予測することを考える。  
セグメント拡張を通して売上に貢献するため、非会員ユーザの属性情報を正確に予測できる予測モデルの構築が求められる。

#### フレームワークに則った予測モデルの学習
**データの観測構造をモデル化する**  
ユーザが20代であるかい否かという年齢に関する情報を目的変数として、それを観測データから予測する問題を考える。  
ユーザ$i$の行動ログなどの特徴量を$X_i$とし、ユーザが20代であるか否かを表す属性情報を$Y$とする。  
仮にユーザの年齢を$age$とするならば目的変数を$Y=I\{age\in [20,30]\}$と定義することもできる。  
今回、ユーザがメディアの会員なのか否かによって属性情報が観測されるか否かが変わってくる問題がある。  
ここでは、ユーザ$i$が会員なのか非会員なのかを表現する$O_i$という2値確率変数を自ら導入することで、  
セグメント拡張におけるデータの観測構造をモデル化することにする。  
ここで、ユーザ$i$が会員である場合は$O_i=1$であり、属性情報がすでに観測されていることを意味する。  
非会員なら$O_i=0$であり、そのユーザの属性情報が観測されていないために予測して埋める必要があることを意味する。  

|ユーザ$i$|会員/非会員|$$O_i$$|年齢|目的変数$$Y_i$$|
|-|-|-|-|-|
|ユーザ1|会員|1|20|1|
|ユーザ2|会員|1|30|0|
|ユーザ3|会員|1|40|0|
|ユーザ4|非会員|0|未観測|未観測|
|ユーザ5|非会員|0|未観測|未観測|

ここでは、$O_i=1$となっているユーザについて観測されている特徴量や目的変数のみを用いて、  
属性情報が観測されておらず$O_i=0$となっているユーザの目的変数を予測することが目的となる。  
学習データとして使う会員データと予測対称となる非会員データは異なる性質を持つデータであると予想できる。  
  
**解きたい問題を特定する**  
セグメント拡張において重要なのは、属性情報をできるだけ正確に予測することである。  
予測するための関数を予測モデル$f_{\phi}$として導入する。  
$f_{\phi}$は、入力変数$X$から目的変数$Y$を予想するためのモデルパラメータ$\phi$を持つ関数である。  
まず、予測モデルの性能を適切に定義し(解きたい問題の特定)、それを最適化する基準でパラメータ$\phi$を得る必要がある。  
官界の問題で予測したいのは、属性情報が観測されていない非会員の目的変数である。  
これを念頭において、次のように予測モデルの真の目的関数を定義する。
$$
J(f_{\phi})=E_X[l(Y,f_{\phi}(X))|O=0]
$$
ここで目的関数$J(f_{\phi})$は、  
「パラメータ$\phi$で定義される予測モデル$f_{\phi}$が、非会員ユーザの属性情報$Y$をどれだけの誤差で予測できているのか?」を定量化したものである。  
この時点では目的関数を理想的に用いたものとして、観測データ$D$とは切り離して定義されている。  
目的関数$J(f_{\phi})$は、次のように変形して表現できる。
$$
\begin{align}
J(f_{\phi})&=E_X[l(Y,f_{\phi}(X))|O=0]\\
&=\int l(Y,f_{\phi}(X))f_{X|O}(x|o=0)dx\\
&=\int l(Y,f_{\phi}(X))\frac{f_X(x)P(O=0|x)}{P(O=0)}dx\\
&\propto \int(1-\theta(x))l(Y,f_{\phi}(X))f_X(x)dx\\
&=E_X[(1-\theta(x))l(Y,f_{\phi}(X))]
\end{align}
$$
特徴量$X$を持つユーザが会員となる確率を、$\theta(X)=E[O|X]=P(O=1|X)$としている。  
このことから、$P(O=0|X)=1-P(O=1|X)=1-\theta(x)$となる。  
目的関数が、非会員である確率$1-\theta(x)$で重みづけされており、非会員になりやすいユーザに対する予測精度を重視する設計となっている。  
予測モデルの学習は、ここで定義した予測誤差を最小化するモデルパラメータ$\phi$を得る問題として定式化される。  
$$
\phi^*=\arg\min_{\phi}J(f_{\phi})
$$
で定義される$\phi^*$が予測誤差を最小にする予測モデルを導くという意味で最適なモデルパラメータである。  
この最適なパラメータによって得られる予測精度$f_{\phi^*}$にできるだけ近い精度を発揮できるパラメータを得ることが目的である。

  
**観測データのみから解きたい問題を近似する**  
今回、目的変数が観測されるのは、すでに会員登録をしているユーザのみであった。  
得られる学習データは、次のように表すことができる。
$$
D=\{(X_i,Y_i)|O_i=1\}_{i=1}^N
$$
非会員ユーザ($O_i=0$)については属性情報が観測されないため、学習データには含まれない。  
ここで、ナイーブな経験損失(観測データにたいする予測誤差の単純平均)は、次のように定義される。
$$
\begin{align}
\hat{J}_{naive}(f_{\phi};D)&=\frac{1}{N}\sum_{i;O_i=1}^Nl(Y_i,f_{\phi}(X_i))\\
&=\frac{1}{N}\sum_{i;O_i=1}^NO_i・l(Y,f_{\phi}(X_i))
\end{align}
$$
ナイーブな経験損失は、目的変数が観測される会員ユーザのみに絞って、予測モデルの予測誤差を計算している。  
ナイーブな経験損失の期待値は次のように計算ができる。
$$
\begin{align}
E_{(X,O)}[\hat{J}_{naive}(f_{\phi};D)]&=E_{(X,O)}[\frac{1}{N}\sum_{i=1}^N O_i・l(Y,f_{\phi}(X_i))]\\
&=\frac{1}{N}\sum_{i=1}^NE_X[E_O[O_i|X_i]・l(Y,f_{\phi}(X_i))]\\
&=\frac{1}{N}\sum_{i=1}^NE_X[\theta(X_i)・l(Y,f_{\phi}(X_i))],\ \theta(X)=E[O|X]\\
&=E_X[\theta(X)・l(Y,f_{\phi}(X_i))]
\end{align}
$$
損失関数が、$\theta(X)$で重み付けされていることが分かる。  
すなわち、ナイーブな経験損失は、会員になりやすいデータを必要以上に重視してしまう設計になっている。  
単に観測されているデータのみをそのまま使うと、非会員に対する予測誤差とはかけ離れた予測損失が計算されてしまうことが分かった。  
  
真の目的関数とナイーブな経験損失の期待値は、それぞれの次のように表される。  
$$
\begin{align}
J(f_{\phi})&=E_X[(1-\theta(X))・l(Y,f_{\phi}(X))]\\
E_{(X,O)}[\hat{J}_{naive}(f_{\phi};D)]&=E_X[\theta(X)・l(Y,f_{\phi}(X))]
\end{align}
$$
これらの間には$(1-\theta(X_i))/\theta(X_i)$だけの乖離が存在する。  
これは、会員ユーザと非会員ユーザの間にある乖離の大きさを定量化する値と言える。  
これを考慮した推定量を新たに設計する。  
$$
\begin{align}
\hat{J}_{IW}(f_{\phi};D)&=\frac{1}{N}\sum_{i:O_i=1}\frac{(1-\theta(X_i))}{\theta(X_i)}・l(Y,f_{\phi}(X))\\
&=\frac{1}{N}\sum_{i=1}O_i・\frac{(1-\theta(X_i))}{\theta(X_i)}・l(Y,f_{\phi}(X))
\end{align}
$$
これを**重要度重み付き損失**と呼ぶこととする。  
$$
\begin{align}
E_{(X,O)}[\hat{J}_{IW}(f_{\phi};D)]&=E_{(X,O)}[\frac{1}{N}\sum_{i=1}^N O_i・\frac{(1-\theta(X_i))}{\theta(X_i)}・l(Y,f_{\phi}(X_i))]\\
&=\frac{1}{N}\sum_{i=1}^NE_X\biggl[E_O[O_i|X_i]・\frac{(1-\theta(X_i))}{\theta(X_i)}・l(Y,f_{\phi}(X_i))\biggr]\\
&=E_X\biggl[\theta(X_i)・\frac{(1-\theta(X_i))}{\theta(X_i)}・l(Y,f_{\phi}(X_i))\biggr]\\
&=E_X\bigl[(1-\theta(X_i))・l(Y,f_{\phi}(X_i))\bigr]\\
&=J(f_{\phi})
\end{align}
$$
重要度重み付け損失は、真の目的関数に対する不変推定量であると言える。  
ここで、重要度重み付け損失の計算に必要なそれぞれのデータが観測される確率$\theta(X)$は、事前にデータから推定しておく必要がある。  
これは、次の誤差最小化問題を解くことで推定できる。  
$$
\hat{\psi}=\arg\min_{\psi}\frac{1}{N}\sum_{i=1}^Nl(O_i,g_{\psi}(X_i))
$$
ここで、$g_{\psi}$は、2値変数$O_i$を予測するためのパラメータ$\psi$で定義される予測モデルである。  
schikit-learnでは次のように実装できる。

In [ ]:
g = LogisticRegression()

# Oに対する予測損失を最小化する基準で予測モデルを学習
g.fit(x_train, o_train)

#学習した予測モデルを用いてthetaを推定
theta_hat = g.predict_proba(x_train)[:, 1]

f = LogisticRegression()

# 重要度重み付け損失を最小化する基準で予測モデルを学習
f.fit(x_train, y_train, sample_weight=((1.0-theta_hat) / theta_hat))

y_pred = f.predict(x_val)

### 2.2 高性能な意思決定を導く
#### 問題設定の導入(広告画像選択)
広告画像選択の例を問題として導入する。  
意思決定システムは、おおよそ次の順序で動作する。  
1. 意思決定のリクエスト(ユーザの訪問)が発生する
2. ユーザの特徴に合わせて適切な広告画像を選択する
3. どの広告画像を表示したかに依存して目的変数(クリックなど)が観測される
4. 1-3を繰り返す

「ユーザの特徴に合わせて適切な広告画像を選択する」部分を自動かつ高性能に遂行できる機械学習モデルを利用可能なデータから学習する。

#### フレームワークに則った意思決定モデルの学習
- KPIを設定する  
ここでは、**広告画像のクリック回数**をKPIとし、これの最大化を目指す。  


- データの観測構造をモデル化する  
ここでは、ユーザ$i$の特徴量を$X_i$で表す。  
意思決定システムがとりうる$K$個の行動集合を、$[K]:=\{1,2,\cdots,K\}$で表す。  
ユーザ$i$に対して選択される行動を$A_i\in[K]$で表し、例えばユーザ$i$に広告画像1が選択される状況は$A_i=1$として表される。  
今回は、あるユーザに対して選択した行動が違えば観測される目的変数も異なる状況を想定している。  
この場合は、**ポテンシャルアウトカムフレームワーク**と呼ばれるモデル化を用いる。  
ポテンシャルアウトカムフレームワークでは、潜在目的変数$Y(・)$という記法を用いる。$(・)$には、取り得る行動が入る。  
ある$a$という行動を選択したならば、その行動に紐づく潜在目的変数$Y(a)$しか観測されず、  
それ以外の行動が仮に選択された場合の潜在目的変数$\{Y(a')|a'\in[K] / \{a\}\}$を観測することはできない。  
$K$個すべての行動に対応する潜在目的変数が背後に存在することを想定しつつも、  
どの行動が選ばれたかに応じて観測される目的変数が切り替わるというイメージである。

- 解きたい問題を特定する  
今回の意思決定の問題を解く方法としては、  
- 機械学習に予測問題を解かせてから、予測値を意思決定に変換する方法  
- 機械学習に意思決定の問題を直接解かせる方法  

の2つが考えられる。  
1つ目の「機械学習に予測問題を解かせてから、予測値を意思決定に変換する方法」では、  
予測モデル$f_{\phi}$により、特徴量$X_i$で表現されるユーザ$i$に対してある行動$a$を選択したときの目的変数を予測する。  
これは、次の目的変数を最小化する基準でパラメータ$\phi$を得る問題として書き表される。  
$$
J(f_{\phi}^{(a)})=E_X[l(Y(a),f_{\phi}^{(a)}(X))],\ a\in[K]
$$